# Experiment LLM using Ollama


### Setup Ollama

In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
    subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [ ]:
!pip install langchain-ollama

### Pull model

In [ ]:
!ollama pull gemma3 # set model

### Prompt categories

In [ ]:
# Zero-shot category
def prompt_zero_shot(puzzle):
    return f"""Los het volgende taalkundige raadsel op. Het bevat drie aanwijzingen en het antwoord is één woord dat op alle drie van toepassing is. Geef het antwoord op de eerste regel en daarna een korte uitleg.

Vraag: {puzzle}. Wat is het?
Antwoord:"""

# One-shot category
def prompt_one_shot(puzzle):
    return f"""Los het volgende taalkundige raadsel op. Het bevat drie aanwijzingen en het antwoord is één woord dat op alle drie van toepassing is. Geef het antwoord op de eerste regel en daarna een korte uitleg. Eerst een voorbeeld, daarna een nieuw raadsel.

Voorbeeld:
Vraag: Het is een onderdeel van een schip, een bevestigingsmiddel, en een gymnastiekoefening. Wat is het?
Antwoord: Schroef.

Nu het raadsel:
Vraag: {puzzle}. Wat is het?
Antwoord:"""

# Three-shot category
def prompt_three_shot(puzzle):
    return f"""Los het volgende taalkundige raadsel op. Het bevat drie aanwijzingen en het antwoord is één woord dat op alle drie van toepassing is. Geef het antwoord op de eerste regel en daarna een korte uitleg. Eerst drie voorbeelden, dan een nieuw raadsel.

Voorbeeld 1:
Vraag: Het is een onderdeel van een schip, een bevestigingsmiddel, en een gymnastiekoefening. Wat is het?
Antwoord: Schroef.

Voorbeeld 2:
Vraag: Het is een winkelketen, een beroep, en een onderdeel van een schip. Wat is het?
Antwoord: Zeeman.

Voorbeeld 3:
Vraag: Het is seksuele anatomie, een beledigend woord, en een vrucht. Wat is het?
Antwoord: Eikel.

Raadsel:
Vraag: {puzzle}. Wat is het?
Antwoord:"""

### Define files

In [ ]:
# File used for experiment
infile = "test_puzzles.txt" # set infile

# File used to save results
outfile = "results_test.txt" # set infile

### Run Ollama

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
import ast


# Read file
def getData(file):
    f = open(file, "r")
    pages = f.readlines()
    return pages

# Write file
def writeData(file, data):
    with open(file, 'w') as f:
        for line in data:
            f.write(f"{line}\n")

# Load data from infile
data = getData(infile)

# Setup model
template = """Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)
model = OllamaLLM(model="gemma3") # set model
chain = prompt | model

results = []
correct = 0
for i, page in enumerate(data):
    load = ast.literal_eval(page)
    puzzle = load["prompt"]
    answer = load["answer"]

    # Set prompt category
    prompt = prompt_three_shot(puzzle) # set shot category

    print(f"Processing: {i+1}/{len(data)}")

    # Generate output
    result = chain.invoke({"question": prompt})

    # Evaluate result
    if answer.lower() in result.lower():
        correct += 1

    results.append({"prompt": prompt, "answer": answer, "result": result})

print(f"Correct: {correct}/{len(data)}")

# Export results
writeData(outfile, results)